In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [15]:
def one_hot_encode(labels):
    # if it is 'setosa' it will be represented as [1, 0, 0] in one_hot_encoder. At a time only one output is hot
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[range(n_labels),labels] = 1
    return one_hot_encode

In [16]:
def read_data():
    data_set = pd.read_csv("heart.dat",header= None,delim_whitespace=True)
    # divide dependent and independent variables
    X = data_set[data_set.columns[0:13]].values
    Y = data_set[data_set.columns[13]]
    # the dependent varibles are being converted to labels
    labelEncoder = LabelEncoder()
    labelEncoder.fit(Y)
    Y = labelEncoder.transform(Y)
    Y = one_hot_encode(Y)
    return (X,Y)

In [17]:
X, Y = read_data()
X, Y = shuffle(X, Y, random_state = 1)
train_x, test_x, train_y, test_y = train_test_split(X, Y, test_size=0.20, random_state=415)

In [18]:
learning_rate = 0.2
training_epochs = 800
no_of_features = X.shape[1]
no_of_outputs = Y.shape[1]


In [19]:
no_of_nodes_in_hidden_layer_1 = 40
no_of_nodes_in_hidden_layer_2 = 50
inputs = tf.placeholder(dtype=tf.float32, shape=[None,no_of_features])
expected_outputs = tf.placeholder(dtype=tf.float32, shape=[None,no_of_outputs])

In [20]:
weights = {
    "level1": tf.Variable(tf.truncated_normal([no_of_features,no_of_nodes_in_hidden_layer_1])),
    "level2": tf.Variable(tf.truncated_normal([no_of_nodes_in_hidden_layer_1,no_of_nodes_in_hidden_layer_2])),
    "out": tf.Variable(tf.truncated_normal([no_of_nodes_in_hidden_layer_2,no_of_outputs]))
}

baises = {
    "level1": tf.Variable(tf.truncated_normal([no_of_nodes_in_hidden_layer_1])),
    "level2": tf.Variable(tf.truncated_normal([no_of_nodes_in_hidden_layer_2])),
    "out": tf.Variable(tf.truncated_normal([no_of_outputs]))
}

In [21]:
# creating saver for our model
saver = tf.train.Saver()

# creating neural net model
def multilayer_perceptron(x, weights, biases):
    # Hidden layer with RELU activations
    layer_1 = tf.add(tf.matmul(x, weights['level1']), biases['level1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    # Hidden layer with sigmoid activations
    layer_2 = tf.add(tf.matmul(layer_1, weights['level2']), biases['level2'])
    layer_2 = tf.nn.sigmoid(layer_2)
    # Output layer with linear activations
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [22]:
predicted_outputs = multilayer_perceptron(inputs, weights, baises)

#creating cost function to reduce to mean cross entropy for optimization
cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= predicted_outputs, labels= expected_outputs))
#creating step function to generate each step to optimize the above cost function
training_step = tf.train.GradientDescentOptimizer(learning_rate = learning_rate).minimize(cost_function)

# creating a global initializer here
init = tf.global_variables_initializer()

In [23]:
# creating tensorflow session
sess = tf.Session()
sess.run(init)
# remember in each session the variables will remain the same

# writing model to file
fileWriter = tf.summary.FileWriter(".",sess.graph)

correct_predictions = tf.equal(tf.argmax(predicted_outputs,1),tf.argmax(expected_outputs,1))
accuracy = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))

In [24]:
# in the below loop, first 3 lines we are training the neural net using train set and then testing the accuracy using test set in remaining steps
for each_epoch in range(training_epochs):
    sess.run(training_step,feed_dict={inputs:train_x,expected_outputs:train_y})
    cost = sess.run(cost_function,feed_dict={inputs:train_x, expected_outputs:train_y})
    pred_outputs = sess.run(predicted_outputs,{inputs:test_x})
    mean_square_error = tf.reduce_mean(tf.square(pred_outputs - test_y))
    mean_square_error_value = sess.run(mean_square_error)
    accuracy_value = sess.run(accuracy,{inputs:test_x, expected_outputs: test_y})
    print "epoch number : "+ str(each_epoch) + " - cost : "+ str(cost)+ " - mse : "+ str(mean_square_error_value)+" - accuracy : "+str(accuracy_value)
    
save_path = saver.save(sess, "./HeartModel")
sess.close()

epoch number : 0 - cost : 1.9424529 - mse : 6.831230505926719 - accuracy : 0.2962963
epoch number : 1 - cost : 1.3456254 - mse : 2.9928192507023006 - accuracy : 0.7037037
epoch number : 2 - cost : 1.2895699 - mse : 3.560742337897695 - accuracy : 0.2962963
epoch number : 3 - cost : 1.3470855 - mse : 2.7856183527542617 - accuracy : 0.7037037
epoch number : 4 - cost : 1.1985253 - mse : 3.742528064748988 - accuracy : 0.2962963
epoch number : 5 - cost : 1.3067229 - mse : 3.118613299527194 - accuracy : 0.7037037
epoch number : 6 - cost : 1.1753246 - mse : 3.79567381729818 - accuracy : 0.2962963
epoch number : 7 - cost : 1.3164513 - mse : 2.6732487785239893 - accuracy : 0.7037037
epoch number : 8 - cost : 1.1501349 - mse : 3.2330342662887794 - accuracy : 0.2962963
epoch number : 9 - cost : 1.222082 - mse : 2.4731092973225968 - accuracy : 0.7037037
epoch number : 10 - cost : 1.3096011 - mse : 3.8842836403634062 - accuracy : 0.2962963
epoch number : 11 - cost : 1.3134863 - mse : 2.6576640993966

epoch number : 100 - cost : 0.81989473 - mse : 2.014434528918012 - accuracy : 0.2777778
epoch number : 101 - cost : 0.8244455 - mse : 1.571701129090732 - accuracy : 0.7037037
epoch number : 102 - cost : 0.8027035 - mse : 1.8923183441249696 - accuracy : 0.2777778
epoch number : 103 - cost : 0.80542445 - mse : 1.527935167412718 - accuracy : 0.7037037
epoch number : 104 - cost : 0.789341 - mse : 1.9112581800288182 - accuracy : 0.2777778
epoch number : 105 - cost : 0.8111889 - mse : 1.4925040589982694 - accuracy : 0.7037037
epoch number : 106 - cost : 0.8051258 - mse : 1.967261444948026 - accuracy : 0.2777778
epoch number : 107 - cost : 0.7930602 - mse : 1.5562716308278188 - accuracy : 0.7037037
epoch number : 108 - cost : 0.7968797 - mse : 1.9462385192642813 - accuracy : 0.2777778
epoch number : 109 - cost : 0.8228635 - mse : 1.552642751955847 - accuracy : 0.7037037
epoch number : 110 - cost : 0.8038422 - mse : 2.0193914628688203 - accuracy : 0.2777778
epoch number : 111 - cost : 0.791138

epoch number : 198 - cost : 0.7300376 - mse : 1.7429628814374256 - accuracy : 0.2962963
epoch number : 199 - cost : 0.7507156 - mse : 1.4611274400538965 - accuracy : 0.7037037
epoch number : 200 - cost : 0.73766214 - mse : 1.7962172130695262 - accuracy : 0.2962963
epoch number : 201 - cost : 0.71795017 - mse : 1.439820369753778 - accuracy : 0.7037037
epoch number : 202 - cost : 0.71167105 - mse : 1.6825455291634137 - accuracy : 0.2962963
epoch number : 203 - cost : 0.7488578 - mse : 1.265902338007366 - accuracy : 0.7037037
epoch number : 204 - cost : 0.7390351 - mse : 1.520580264896436 - accuracy : 0.2962963
epoch number : 205 - cost : 0.7393474 - mse : 1.2223958038027465 - accuracy : 0.7037037
epoch number : 206 - cost : 0.7469861 - mse : 1.684207008092824 - accuracy : 0.2962963
epoch number : 207 - cost : 0.76382905 - mse : 1.2219310045507203 - accuracy : 0.7037037
epoch number : 208 - cost : 0.7664703 - mse : 1.611465090848201 - accuracy : 0.2962963
epoch number : 209 - cost : 0.759

epoch number : 291 - cost : 0.6277312 - mse : 1.4268331553256022 - accuracy : 0.7222222
epoch number : 292 - cost : 0.6257356 - mse : 1.4921942472118772 - accuracy : 0.46296296
epoch number : 293 - cost : 0.6309007 - mse : 1.445418725709017 - accuracy : 0.7037037
epoch number : 294 - cost : 0.62634397 - mse : 1.5025770566697607 - accuracy : 0.44444445
epoch number : 295 - cost : 0.62576866 - mse : 1.4651028208675834 - accuracy : 0.7037037
epoch number : 296 - cost : 0.6345681 - mse : 1.5809026255177314 - accuracy : 0.33333334
epoch number : 297 - cost : 0.65099055 - mse : 1.4247461299624722 - accuracy : 0.7222222
epoch number : 298 - cost : 0.64555997 - mse : 1.5845016761658242 - accuracy : 0.42592594
epoch number : 299 - cost : 0.6486443 - mse : 1.359007474608803 - accuracy : 0.7037037
epoch number : 300 - cost : 0.6363628 - mse : 1.5239596461199416 - accuracy : 0.46296296
epoch number : 301 - cost : 0.63007927 - mse : 1.447796732213801 - accuracy : 0.7037037
epoch number : 302 - cost

epoch number : 386 - cost : 0.6236622 - mse : 1.5618541786528883 - accuracy : 0.7222222
epoch number : 387 - cost : 0.6186612 - mse : 1.5168212189065318 - accuracy : 0.7222222
epoch number : 388 - cost : 0.6229581 - mse : 1.5875932666294463 - accuracy : 0.7222222
epoch number : 389 - cost : 0.6167022 - mse : 1.5075090962692912 - accuracy : 0.7222222
epoch number : 390 - cost : 0.6176484 - mse : 1.4983747420110598 - accuracy : 0.7222222
epoch number : 391 - cost : 0.6164157 - mse : 1.502312452022762 - accuracy : 0.7222222
epoch number : 392 - cost : 0.6145665 - mse : 1.5087002260590212 - accuracy : 0.7037037
epoch number : 393 - cost : 0.6170239 - mse : 1.5027631035194475 - accuracy : 0.7222222
epoch number : 394 - cost : 0.6158642 - mse : 1.513563674829268 - accuracy : 0.7222222
epoch number : 395 - cost : 0.61918896 - mse : 1.5088596904795621 - accuracy : 0.7222222
epoch number : 396 - cost : 0.6168352 - mse : 1.4848667888563545 - accuracy : 0.7222222
epoch number : 397 - cost : 0.615

epoch number : 481 - cost : 0.7963633 - mse : 1.5248698371054696 - accuracy : 0.7222222
epoch number : 482 - cost : 0.7748748 - mse : 2.0583781495930284 - accuracy : 0.2962963
epoch number : 483 - cost : 0.7144194 - mse : 1.598070956848192 - accuracy : 0.7037037
epoch number : 484 - cost : 0.6796666 - mse : 1.7972610192518987 - accuracy : 0.2962963
epoch number : 485 - cost : 0.656963 - mse : 1.5221569248203155 - accuracy : 0.7222222
epoch number : 486 - cost : 0.6416753 - mse : 1.5670917379703828 - accuracy : 0.5555556
epoch number : 487 - cost : 0.6420823 - mse : 1.4853817897071533 - accuracy : 0.7037037
epoch number : 488 - cost : 0.64307487 - mse : 1.5972501728540303 - accuracy : 0.5
epoch number : 489 - cost : 0.63138646 - mse : 1.3655058102268016 - accuracy : 0.7222222
epoch number : 490 - cost : 0.6630054 - mse : 1.487763121385734 - accuracy : 0.5925926
epoch number : 491 - cost : 0.693803 - mse : 1.306922510882183 - accuracy : 0.7222222
epoch number : 492 - cost : 0.6836118 - m

epoch number : 574 - cost : 0.6957375 - mse : 1.3351311561268961 - accuracy : 0.2962963
epoch number : 575 - cost : 0.68017495 - mse : 1.0558774082278737 - accuracy : 0.7222222
epoch number : 576 - cost : 0.6601513 - mse : 1.2414526665966878 - accuracy : 0.2962963
epoch number : 577 - cost : 0.6594126 - mse : 1.0276620692489586 - accuracy : 0.7037037
epoch number : 578 - cost : 0.65378046 - mse : 1.2638125534893003 - accuracy : 0.2962963
epoch number : 579 - cost : 0.6446765 - mse : 1.0521822475040628 - accuracy : 0.7222222
epoch number : 580 - cost : 0.668765 - mse : 1.3058596772605016 - accuracy : 0.2962963
epoch number : 581 - cost : 0.6622501 - mse : 1.0887862709167937 - accuracy : 0.7222222
epoch number : 582 - cost : 0.7021997 - mse : 1.3395954308885112 - accuracy : 0.2962963
epoch number : 583 - cost : 0.7191432 - mse : 1.0678241200180867 - accuracy : 0.7407407
epoch number : 584 - cost : 0.66893995 - mse : 1.2785549714215731 - accuracy : 0.2962963
epoch number : 585 - cost : 0.

epoch number : 667 - cost : 0.6227921 - mse : 1.1578590336917107 - accuracy : 0.7037037
epoch number : 668 - cost : 0.63689625 - mse : 1.2899119303706168 - accuracy : 0.46296296
epoch number : 669 - cost : 0.63274395 - mse : 1.1762806176198564 - accuracy : 0.7222222
epoch number : 670 - cost : 0.61948955 - mse : 1.2759624557816662 - accuracy : 0.5185185
epoch number : 671 - cost : 0.6239926 - mse : 1.1611102593004896 - accuracy : 0.7037037
epoch number : 672 - cost : 0.6282824 - mse : 1.2579192693643226 - accuracy : 0.5185185
epoch number : 673 - cost : 0.6228405 - mse : 1.0818404798492458 - accuracy : 0.7407407
epoch number : 674 - cost : 0.61768395 - mse : 1.2134259014090618 - accuracy : 0.5
epoch number : 675 - cost : 0.6273168 - mse : 1.0703500721136943 - accuracy : 0.7037037
epoch number : 676 - cost : 0.71468705 - mse : 1.518977302112671 - accuracy : 0.44444445
epoch number : 677 - cost : 0.70359015 - mse : 1.2849644784110206 - accuracy : 0.7037037
epoch number : 678 - cost : 0.7

epoch number : 762 - cost : 0.5974901 - mse : 1.0988025376605053 - accuracy : 0.6296296
epoch number : 763 - cost : 0.6066803 - mse : 1.068243175084982 - accuracy : 0.6666667
epoch number : 764 - cost : 0.602402 - mse : 1.1010366747512448 - accuracy : 0.6296296
epoch number : 765 - cost : 0.59904355 - mse : 1.1550428484703403 - accuracy : 0.6666667
epoch number : 766 - cost : 0.59790456 - mse : 1.1625816464930063 - accuracy : 0.6296296
epoch number : 767 - cost : 0.60275966 - mse : 1.2100808046571112 - accuracy : 0.6296296
epoch number : 768 - cost : 0.59937596 - mse : 1.242149409534357 - accuracy : 0.6296296
epoch number : 769 - cost : 0.6024112 - mse : 1.2012870347220486 - accuracy : 0.6481481
epoch number : 770 - cost : 0.60127854 - mse : 1.2170936452842365 - accuracy : 0.6296296
epoch number : 771 - cost : 0.6012021 - mse : 1.2362218453428808 - accuracy : 0.6296296
epoch number : 772 - cost : 0.6004424 - mse : 1.1424369344746172 - accuracy : 0.6481481
epoch number : 773 - cost : 0.